In [1]:
#test using BlackstoneNLP

#import standard library modules
import sys
from collections import Counter
from typing import List, Any

#modules from the community
import spacy
from dataclasses import dataclass #backported this module from 3.7

#import lxml

from spacy import displacy
from blackstone.displacy_palette import ner_displacy_options

#blackstone improved citations
from blackstone.pipeline.sentence_segmenter import SentenceSegmenter
from blackstone.rules import CITATION_PATTERNS

import en_core_web_sm


#BeautifulSoup modules

In [2]:
nlp = spacy.load('en_blackstone_proto') #the blackstone model
nlp2 = en_core_web_sm.load()
#load sentence segmenter
sentence_segmenter = SentenceSegmenter(nlp.vocab, CITATION_PATTERNS)
nlp.add_pipe(sentence_segmenter, before="parser")

#nlp=spacy.load('en_core_web_sm') #the default spacy model

In [3]:
with open('nfib_v_sebelius.txt', 'r') as in_file:
    nfib_data = in_file.read()

#with open('us_v_gilman.json', 'r') as in_file:
#    gilman = json.load(in_file)



In [4]:
bx = nlp(nfib_data)
lay_doc = nlp2(nfib_data)

In [5]:
#for ent in bx.ents:
#    print(ent)
#display entities for both documents

#when we have x and y, we create a new example of something to train the model
#and we feed it into the model

<b>In the section above we can see the law's text displayed using displaCy. We have highlighted the cases, provisions, laws, etc in the text.</b>


In [6]:
#display_casename_citations_filtered(doc)
def get_casename_citations_filtered(doc):
    #one
    '''Takes a spacy doc object and returns a dictionary of cases using the blackstone nlp model
    args:
        doc: the spacy doc object
    returns:
        
    '''
    cases = (i for i in doc.ents if i.label_ == 'CASENAME')
    actual_cases=[]
    results = {case.text:[item for item in case] for case in cases}
    return results

def get_actual_cases(case_list: dict) -> List:
    '''Takes the cases and removes some of the ones that are not cases
    like the ones without a v in them'''
    actual_cases = []
    for k, v in case_list.items():
        for i in v:
            if i.pos_ == 'ADP' and i.dep_ == 'prep':
                actual_cases.append(k)
    return actual_cases

def get_cases_from_doc(doc):
    return get_actual_cases(get_casename_citations_filtered(doc))

In [7]:
result_one = get_cases_from_doc(bx)
result_one

#make a dictionary with the text, and a retokenized version of the result
result_two = {i: [j for j in nlp2(i)] for i in result_one}

barred_words = {'brief', 'financial outlook'}
barred_pos = {'NUM'}

clean_cases = []
for i, j in result_two.items():
   #print(i)
    #print(i, [(token.text, token.pos_, token.dep_) for token in j])
    for token in j:
        #eliminate the ones with numbers
        if token.pos_ not in barred_pos:
            if token.text.lower() not in barred_words:
                clean_cases.append(i)
            elif token.text.lower() in barred_words:
                if token.dep_ != 'ROOT':
                    clean_cases.append(i)

case_counter = Counter(clean_cases)
#The number of times each case is cited
print(case_counter)
#print(clean_cases)
clean_cases = list(set([str(i) for i in clean_cases]))
print(clean_cases)

Counter({'Office of Management and Budget, Historical Tables, Budget of\n': 12, 'Heart of Atlanta Motel, Inc. v. United\n': 9, 'American Power & Light Co. v. SEC,\n': 9, 'Heart of Atlanta Motel, Inc. v. United States': 9, 'Table 12.3—Total Outlays for Grants to State and Local Gov-': 9, 'United States v. Detroit Timber & Lumber Co.': 8, 'Ayotte v. Planned Parenthood of Northern New Eng': 8, 'College Savings Bank v. Florida Prepaid Postsecond-\n': 8, 'Center for Applied Ethics, Voluntary Health\n': 8, 'United States v. Reorganized CF&I Fabricators of Utah': 8, 'Brief for National Health Law Program et al.': 8, 'Metropolitan Washington Airports Authority v. Citizens for\n': 8, 'Temporary Assistance for Needy Families (TANF)': 8, 'Enochs v. Williams Packing & Nav.': 7, 'Department of Revenue of Mont. v.\n': 7, 'Nelson v. Sears, Roebuck & Co.': 7, 'Cherokee Nation v. Southern Kansas R. Co.': 7, 'Perez v. United States, O. T.': 7, 'Bowen v. Public Agencies Opposed to\n': 7, 'NLRB v. Jones &

In [8]:
#def remove_statues_from_cases(case_list: List, law_doc: Any, lay_doc: Any):
#    '''Takes a list of cases and removes the ones that are actually statutes'''
#    case_dict = {case: []}
#    pass
print(result_two)

{'United States v. Detroit Timber & Lumber Co.': [United, States, v., Detroit, Timber, &, Lumber, Co.], 'United States v. Lopez': [United, States, v., Lopez], 'Hooper v. California': [Hooper, v., California], 'Crowell v. Benson': [Crowell, v., Benson], 'Bailey v. Drexel\n    Furniture Co.': [Bailey, v., Drexel, 
    , Furniture, Co.], 'New York v. United States,': [New, York, v., United, States, ,], 'Ayotte v. Planned Parenthood of Northern New Eng': [Ayotte, v., Planned, Parenthood, of, Northern, New, Eng], 'Gibbons v. Ogden': [Gibbons, v., Ogden], 'United States v.\n': [United, States, v., 
], 'New York v. United\n': [New, York, v., United, 
], 'Wickard v. Filburn': [Wickard, v., Filburn], 'Perez v.\n': [Perez, v., 
], 'College Savings Bank v. Florida Prepaid Postsecond-\n': [College, Savings, Bank, v., Florida, Prepaid, Postsecond-, 
], 'South Dakota v. Dole': [South, Dakota, v., Dole], 'United States v. Harris': [United, States, v., Harris], 'Thomas More Law Center v.\n': [Thomas, 

In [68]:
judges = [i for i in get_judges(bx, doc)]
for i in judges:
    for j in i:
        print(i.text, [j.text for j in i], end='\t')

NameError: name 'get_judges' is not defined

In [ ]:
def remove_statutes(case_list: List):
    '''removes statutes, each of which have a numeric elemnt'''

In [ ]:
#how to get the cases
#get_actual_cases(get_casename_citations_filtered(doc))

In [ ]:
cases_found = get_cases_from_doc(doc)
print('Cases found: ', [(i, i.label_) for i in cases_found])

In [21]:
#print(dir(spacy.tokens.Token))
#for i in cases_found:
#    for token in i:
#        print(token.text, token.pos_, token.dep_, [i for i in token.lefts], [i for i in token.rights])

In [385]:
@dataclass
class CaseWithNLP:
    case_name: str
    name_token: spacy.tokens.Token
    plaintiff: spacy.tokens.Token
    respondant: spacy.tokens.Token
    

In [388]:
def get_case_list(doc):
    result = get_cases_from_doc(doc)
    return result

In [390]:
res = make_case_with_nlp(doc)
print(res)

[United States v. Standard Oil Co., Dalehite v. United States,]


In [ ]:
def 